# Komorbiditäts-Netzwerk Analyse
#### Ausfügung des Notebooks über Kernel-Restart; Bei weniger als 1000 Patienten nicht Empfohlen!; Synthea nicht deterministisch andere Ergebisse der Analysen
## Synthea Healthcare Demo - Neo4j

**Zweck:** Analyse von Komorbiditätsmustern in synthetischen Patientendaten

**Was sind Komorbiditäten?** Wenn Patienten gleichzeitig mehrere Erkrankungen haben (z.B. Diabetes und Bluthochdruck), spricht man von Komorbiditäten. Die Analyse dieser Muster hilft, Risikopatienten zu identifizieren und Behandlungsstrategien zu optimieren.

**Datensatz:** Synthetische Patientendaten generiert mit Synthea™ Patient Generator (Anzahl variiert je nach Generierung: 25-2500 Patienten möglich)

**Analysen in diesem Notebook:**
- Datenbank-Statistiken: Überblick über Patienten, Erkrankungen und Beziehungen
- Häufigste Komorbiditätspaare: Welche Krankheiten treten zusammen auf?
- Krankheitsprävalenz: Wie verbreitet sind chronische Erkrankungen?
- Soziale Determinanten: Einfluss von sozialen Faktoren auf Gesundheit
- Multimorbidität: Wie viele Patienten haben mehrere chronische Erkrankungen?

**Datum:** November 17, 2025


## Setup: Verbindung zu Neo4j herstellen

In [1]:
from neo4j import GraphDatabase
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Verbindungseinstellungen
NEO4J_URI = "bolt://neo4j-synthea:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "synthea123"

# Driver erstellen
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def run_query(query):
    """Führt eine Cypher-Query aus und gibt Ergebnisse als DataFrame zurück"""
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([dict(record) for record in result])

print("✅ Verbindung zu Neo4j hergestellt")


✅ Verbindung zu Neo4j hergestellt


## 1. Datenbank-Überblick

Zuerst verschaffen wir uns einen Überblick über die vorhandenen Daten.

In [2]:
# Holen Sie sich zunächst die Gesamtanzahl der Patienten für Berechnungen
patient_count_query = "MATCH (p:Patient) RETURN count(p) as total"
total_patients = run_query(patient_count_query).iloc[0]['total']

query = """
MATCH (p:Patient) WITH count(p) as patients
MATCH (c:Condition) WITH patients, count(c) as conditions
MATCH (cd:ChronicDisease) WITH patients, conditions, count(cd) as chronic_diseases
MATCH (sd:SocialDeterminant) WITH patients, conditions, chronic_diseases, count(sd) as social_determinants
MATCH ()-[co:CO_OCCURS_WITH]->() WITH patients, conditions, chronic_diseases, social_determinants, count(co) as comorbidities
MATCH ()-[pc:HAS_CONDITION]->() 
RETURN patients, conditions, chronic_diseases, social_determinants, comorbidities, count(pc) as patient_conditions
"""

stats = run_query(query).iloc[0]

print("=" * 60)
print("DATENBANK-STATISTIKEN")
print("=" * 60)
print(f"Anzahl Patienten:                {stats['patients']:,}")
print(f"Anzahl Erkrankungen:             {stats['conditions']:,}")
print(f"  - Chronische Erkrankungen:     {stats['chronic_diseases']:,}")
print(f"  - Soziale Determinanten:       {stats['social_determinants']:,}")
print(f"Patient-Erkrankungs-Beziehungen: {stats['patient_conditions']:,}")
print(f"Komorbiditäts-Beziehungen:       {stats['comorbidities']:,}")
print("=" * 60)


DATENBANK-STATISTIKEN
Anzahl Patienten:                2,856
Anzahl Erkrankungen:             272
  - Chronische Erkrankungen:     178
  - Soziale Determinanten:       10
Patient-Erkrankungs-Beziehungen: 100,403
Komorbiditäts-Beziehungen:       3,358


## 2. Top 20 Komorbiditätspaare

Diese Analyse zeigt, welche Krankheiten besonders häufig zusammen auftreten. Solche Muster sind wichtig für die Behandlungsplanung und Risikoeinschätzung.

In [3]:
query = """
MATCH (c1:Condition)-[r:CO_OCCURS_WITH]->(c2:Condition)
RETURN c1.description as Krankheit_1,
       c2.description as Krankheit_2,
       coalesce(r.cooccurrenceCount, r.weight) as Anzahl_Patienten
ORDER BY coalesce(r.cooccurrenceCount, r.weight) DESC
LIMIT 20
"""

comorbidities = run_query(query)
print("\nTOP 20 KOMORBIDITÄTSPAARE\n")
print(comorbidities.to_string(index=False))



TOP 20 KOMORBIDITÄTSPAARE

                            Krankheit_1                                                            Krankheit_2  Anzahl_Patienten
                  Gingivitis (disorder)                                            Gingival disease (disorder)              1258
Body mass index 30+ - obesity (finding)                                                  Gingivitis (disorder)              1227
                      Anemia (disorder)                                                  Gingivitis (disorder)               937
Body mass index 30+ - obesity (finding)                                            Gingival disease (disorder)               722
Body mass index 30+ - obesity (finding)                                                      Anemia (disorder)               687
                  Gingivitis (disorder)                                      Essential hypertension (disorder)               614
                      Anemia (disorder)                              

## 3. Häufigste chronische Erkrankungen

**Was sind chronische Erkrankungen?** Langfristige Gesundheitsprobleme, die kontinuierliche medizinische Betreuung erfordern (z.B. Diabetes, Bluthochdruck). Die Prävalenz zeigt, wie viele Patienten im Datensatz betroffen sind.

In [4]:
query = f"""
MATCH (p:Patient)-[:HAS_CONDITION]->(c:ChronicDisease)
WITH c, count(DISTINCT p) as patient_count
RETURN c.description as Erkrankung,
       patient_count as Anzahl_Patienten,
       round(100.0 * patient_count / {total_patients}, 1) as Prävalenz_Prozent
ORDER BY patient_count DESC
LIMIT 20
"""

diseases = run_query(query)
print("\nTOP 20 CHRONISCHE ERKRANKUNGEN NACH PRÄVALENZ\n")
print(diseases.to_string(index=False))



TOP 20 CHRONISCHE ERKRANKUNGEN NACH PRÄVALENZ

                                                            Erkrankung  Anzahl_Patienten  Prävalenz_Prozent
                                                 Gingivitis (disorder)              2318               81.2
                                           Gingival disease (disorder)              1331               46.6
                               Body mass index 30+ - obesity (finding)              1319               46.2
                                                     Anemia (disorder)              1023               35.8
                                     Essential hypertension (disorder)               638               22.3
Abnormal findings diagnostic imaging heart+coronary circulat (finding)               473               16.6
                                     Ischemic heart disease (disorder)               472               16.5
                                       Metabolic syndrome X (disorder)               391

## 4. Metabolisches Syndrom - Cluster-Analyse

**Was ist das metabolische Syndrom?** Eine Kombination aus Übergewicht, Bluthochdruck, erhöhtem Blutzucker und Fettstoffwechselstörungen. Diese Erkrankungen treten häufig gemeinsam auf und erhöhen das Risiko für Herzinfarkt und Schlaganfall deutlich.

In [5]:
query = """
MATCH (c:ChronicDisease)<-[:HAS_CONDITION]-(p:Patient)
WHERE c.description IN [
    'Body mass index 30+ - obesity (finding)',
    'Essential hypertension (disorder)',
    'Metabolic syndrome X (disorder)',
    'Diabetes mellitus type 2 (disorder)',
    'Anemia (disorder)',
    'Hyperlipidemia (disorder)'
]
WITH c, count(DISTINCT p) as patients
OPTIONAL MATCH (c)-[r:CO_OCCURS_WITH]-(other:ChronicDisease)
WHERE other.description IN [
    'Body mass index 30+ - obesity (finding)',
    'Essential hypertension (disorder)',
    'Metabolic syndrome X (disorder)',
    'Diabetes mellitus type 2 (disorder)',
    'Hyperlipidemia (disorder)'
]
WITH c, patients, count(r) as connections
RETURN c.description as Erkrankung,
       patients as Anzahl_Patienten,
       connections as Verbindungen_im_Cluster
ORDER BY patients DESC
"""

metabolic = run_query(query)
print("\nMETABOLISCHES SYNDROM - CLUSTER\n")
print(metabolic.to_string(index=False))



METABOLISCHES SYNDROM - CLUSTER

                             Erkrankung  Anzahl_Patienten  Verbindungen_im_Cluster
Body mass index 30+ - obesity (finding)              1319                        4
                      Anemia (disorder)              1023                        5
      Essential hypertension (disorder)               638                        4
        Metabolic syndrome X (disorder)               391                        4
              Hyperlipidemia (disorder)               294                        4
    Diabetes mellitus type 2 (disorder)               196                        4


## 5. Soziale Determinanten und ihr Einfluss

**Was sind soziale Determinanten?** Faktoren wie Wohnsituation, Bildung, Einkommen oder soziale Isolation, die die Gesundheit beeinflussen. Sie können Risikofaktoren für Erkrankungen sein oder die Behandlung erschweren.

In [6]:
query = f"""
MATCH (s:SocialDeterminant)<-[:HAS_CONDITION]-(p:Patient)
WITH s, count(DISTINCT p) as patient_count
RETURN s.description as Sozialer_Faktor,
       patient_count as Anzahl_Patienten,
       round(100.0 * patient_count / {total_patients}, 1) as Prävalenz_Prozent
ORDER BY patient_count DESC
LIMIT 15
"""

social = run_query(query)
print("\nTOP 15 SOZIALE DETERMINANTEN\n")
print(social.to_string(index=False))



TOP 15 SOZIALE DETERMINANTEN

                                 Sozialer_Faktor  Anzahl_Patienten  Prävalenz_Prozent
                  Full-time employment (finding)              2244               78.6
                  Part-time employment (finding)              1831               64.1
                    Not in labor force (finding)              1389               48.6
             Received higher education (finding)              1344               47.1
      Victim of intimate partner abuse (finding)              1227               43.0
Reports of violence in the environment (finding)               913               32.0
                 Has a criminal record (finding)               552               19.3
      Lack of access to transportation (finding)               274                9.6
Only received primary school education (finding)               255                8.9
                  Respiratory distress (finding)                34                1.2


## 6. Multimorbidität - Patienten mit mehreren chronischen Erkrankungen

**Was ist Multimorbidität?** Wenn Patienten zwei oder mehr chronische Erkrankungen gleichzeitig haben. Diese Patienten benötigen besondere Aufmerksamkeit, da Wechselwirkungen zwischen Medikamenten und komplexere Behandlungspläne berücksichtigt werden müssen.

In [7]:
query = f"""
MATCH (p:Patient)-[:HAS_CONDITION]->(c:ChronicDisease)
WITH p, count(DISTINCT c) as chronic_count
WHERE chronic_count >= 2
WITH chronic_count, count(p) as patient_count
RETURN chronic_count as Anzahl_chronischer_Erkrankungen,
       patient_count as Anzahl_Patienten,
       round(100.0 * patient_count / {total_patients}, 1) as Prozent
ORDER BY chronic_count DESC
LIMIT 20
"""

multi_morbidity = run_query(query)
print("\nMULTIMORBIDITÄT - VERTEILUNG\n")
print(multi_morbidity.to_string(index=False))
print(f"\nPatienten mit 2+ chronischen Erkrankungen: {multi_morbidity['Anzahl_Patienten'].sum():,}")
print(f"Prozentsatz: {multi_morbidity['Prozent'].sum():.1f}%")



MULTIMORBIDITÄT - VERTEILUNG

 Anzahl_chronischer_Erkrankungen  Anzahl_Patienten  Prozent
                              33                 1      0.0
                              30                 1      0.0
                              29                 1      0.0
                              27                 2      0.1
                              26                 3      0.1
                              25                 6      0.2
                              24                 1      0.0
                              23                10      0.4
                              22                12      0.4
                              21                10      0.4
                              20                18      0.6
                              19                22      0.8
                              18                21      0.7
                              17                27      0.9
                              16                36      1.3
         

## 7. Detailanalyse: Anämie und Adipositas (Übergewicht)

**Warum diese Kombination?** Anämie (Blutarmut) und Adipositas (starkes Übergewicht, BMI > 30) treten häufig gemeinsam auf. Diese Analyse zeigt, wie stark diese beiden Erkrankungen miteinander assoziiert sind - ein wichtiges Muster für die Behandlungsplanung.

In [8]:
query = """
// Schritt 1: Zähle Patienten mit beiden Erkrankungen
MATCH (p:Patient)-[:HAS_CONDITION]->(anemia:ChronicDisease {description: 'Anemia (disorder)'}),
      (p)-[:HAS_CONDITION]->(obesity:ChronicDisease {description: 'Body mass index 30+ - obesity (finding)'})
WITH count(DISTINCT p) as both_count

// Schritt 2: Zähle Anämie gesamt (propagiere both_count)
MATCH (p_anemia:Patient)-[:HAS_CONDITION]->(:ChronicDisease {description: 'Anemia (disorder)'})
WITH both_count, count(DISTINCT p_anemia) as anemia_total

// Schritt 3: Zähle Adipositas gesamt (propagiere both_count + anemia_total)
MATCH (p_obesity:Patient)-[:HAS_CONDITION]->(:ChronicDisease {description: 'Body mass index 30+ - obesity (finding)'})
WITH both_count, anemia_total, count(DISTINCT p_obesity) as obesity_total

RETURN both_count as Beide_Erkrankungen,
       anemia_total as Anämie_Gesamt,
       obesity_total as Adipositas_Gesamt,
       round(100.0 * both_count / anemia_total, 1) as Prozent_von_Anämie,
       round(100.0 * both_count / obesity_total, 1) as Prozent_von_Adipositas
"""

result = run_query(query)
if result.empty:
    print("\n⚠️ Keine Daten gefunden - prüfen Sie, ob Anämie/Adipositas in ChronicDisease existieren")
else:
    analysis = result.iloc[0]
    print("\nANÄMIE ←→ ADIPOSITAS ANALYSE\n")
    print(f"Patienten mit BEIDEN Erkrankungen:          {analysis['Beide_Erkrankungen']:,}")
    print(f"Patienten mit Anämie gesamt:                {analysis['Anämie_Gesamt']:,}")
    print(f"Patienten mit Adipositas gesamt:            {analysis['Adipositas_Gesamt']:,}")
    print(f"% von Anämie-Patienten mit Adipositas:      {analysis['Prozent_von_Anämie']:.1f}%")
    print(f"% von Adipositas-Patienten mit Anämie:      {analysis['Prozent_von_Adipositas']:.1f}%")



ANÄMIE ←→ ADIPOSITAS ANALYSE

Patienten mit BEIDEN Erkrankungen:          687.0
Patienten mit Anämie gesamt:                1,023.0
Patienten mit Adipositas gesamt:            1,319.0
% von Anämie-Patienten mit Adipositas:      67.2%
% von Adipositas-Patienten mit Anämie:      52.1%


## Abschluss: Verbindung schließen

In [9]:
driver.close()
print("✅ Verbindung geschlossen")


✅ Verbindung geschlossen
